In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm_notebook

In [16]:
movie = tmdb.Movies(basics)

In [17]:
info = movie.info()
info

APIKeyError: 

In [2]:
import os, time, json
import tmdbsimple as tmdb
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_reuslts_2001.json']

In [3]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [4]:
basics = pd.read_csv('Data/title_basics.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0079644,movie,November 1828,November 1828,0,2001.0,NaN,140,"Drama,War"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [5]:
YEARS_TO_GET = [2000,2001]

In [6]:
errors = []

In [7]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS',position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_reuslts_{YEAR}.json'

file_exists = os.path.isfile(JSON_FILE)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
if file_exists == False:
# save an empty dict with just "imdb_id" to the new json file.
    with open(JSON_FILE,'w') as f:
        json.dump([{'imdb_id':0}],f)

In [9]:
df = basics.loc[basics['startYear']==YEAR].copy()

movie_ids = df['tconst'].copy()
df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
3,tt0079644,movie,November 1828,November 1828,0,2001.0,NaN,140,"Drama,War"
5,tt0089067,movie,El día de los albañiles 2,El día de los albañiles 2,0,2001.0,NaN,90,Comedy
17,tt0114447,movie,The Silent Force,The Silent Force,0,2001.0,NaN,90,Action
18,tt0114722,movie,3 noches,3 noches,0,2001.0,NaN,105,"Crime,Thriller"


In [10]:
previous_df = pd.read_json(JSON_FILE)

In [11]:
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

In [12]:
    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])

Movies from 2001:   0%|          | 0/2844 [00:00<?, ?it/s]

In [13]:
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz",
                    compression="gzip",index=False)

In [14]:
print(f"- Total errors: {len(errors)}")

- Total errors: 2844
